In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

FILEMAP = {
    # ───── A ─────
    'ADA'  : r'../resources/data/raw/ADA_USDT_1m.csv',
    'ALGO' : r'../resources/data/raw/ALGO_USDT_1m.csv',
    'ANKR' : r'../resources/data/raw/ANKR_USDT_1m.csv',
    'ATOM' : r'../resources/data/raw/ATOM_USDT_1m.csv',
    # ───── B ─────
    'BAT'  : r'../resources/data/raw/BAT_USDT_1m.csv',
    'BNB'  : r'../resources/data/raw/BNB_USDT_1m.csv',
    'BTC'  : r'../resources/data/raw/BTC_USDT_1m.csv',
    # ───── C ─────
    'CELR' : r'../resources/data/raw/CELR_USDT_1m.csv',
    'CHZ'  : r'../resources/data/raw/CHZ_USDT_1m.csv',
    'COS'  : r'../resources/data/raw/COS_USDT_1m.csv',
    'CVC'  : r'../resources/data/raw/CVC_USDT_1m.csv',
    # ───── D ─────
    'DASH' : r'../resources/data/raw/DASH_USDT_1m.csv',
    'DENT' : r'../resources/data/raw/DENT_USDT_1m.csv',
    'DOCK' : r'../resources/data/raw/DOCK_USDT_1m.csv',
    'DOGE' : r'../resources/data/raw/DOGE_USDT_1m.csv',
    'DUSK' : r'../resources/data/raw/DUSK_USDT_1m.csv',
    # ───── E ─────
    'ENJ'  : r'../resources/data/raw/ENJ_USDT_1m.csv',
    'EOS'  : r'../resources/data/raw/EOS_USDT_1m.csv',
    'ETC'  : r'../resources/data/raw/ETC_USDT_1m.csv',
    'ETH'  : r'../resources/data/raw/ETH_USDT_1m.csv',
    # ───── F ─────
    'FET'  : r'../resources/data/raw/FET_USDT_1m.csv',
    'FTM'  : r'../resources/data/raw/FTM_USDT_1m.csv',
    'FUN'  : r'../resources/data/raw/FUN_USDT_1m.csv',
    # ───── H ─────
    'HOT'  : r'../resources/data/raw/HOT_USDT_1m.csv',
    # ───── I ─────
    'ICX'  : r'../resources/data/raw/ICX_USDT_1m.csv',
    'IOST' : r'../resources/data/raw/IOST_USDT_1m.csv',
    'IOTA' : r'../resources/data/raw/IOTA_USDT_1m.csv',
    # ───── K ─────
    'KEY'  : r'../resources/data/raw/KEY_USDT_1m.csv',
    # ───── L ─────
    'LINK' : r'../resources/data/raw/LINK_USDT_1m.csv',
    'LTC'  : r'../resources/data/raw/LTC_USDT_1m.csv',
    # ───── M ─────
    'MATIC': r'../resources/data/raw/MATIC_USDT_1m.csv',
    'MTL'  : r'../resources/data/raw/MTL_USDT_1m.csv',
    # ───── N ─────
    'NEO'  : r'../resources/data/raw/NEO_USDT_1m.csv',
    'NULS' : r'../resources/data/raw/NULS_USDT_1m.csv',
    # ───── O ─────
    'OMG'  : r'../resources/data/raw/OMG_USDT_1m.csv',
    'ONE'  : r'../resources/data/raw/ONE_USDT_1m.csv',
    'ONG'  : r'../resources/data/raw/ONG_USDT_1m.csv',
    'ONT'  : r'../resources/data/raw/ONT_USDT_1m.csv',
    # ───── P ─────
    'PERL' : r'../resources/data/raw/PERL_USDT_1m.csv',
    # ───── Q ─────
    'QTUM' : r'../resources/data/raw/QTUM_USDT_1m.csv',
    # ───── T ─────
    'TFUEL': r'../resources/data/raw/TFUEL_USDT_1m.csv',
    'THETA': r'../resources/data/raw/THETA_USDT_1m.csv',
    'TOMO' : r'../resources/data/raw/TOMO_USDT_1m.csv',
    'TRX'  : r'../resources/data/raw/TRX_USDT_1m.csv',
    'TUSD' : r'../resources/data/raw/TUSD_USDT_1m.csv',
    # ───── U ─────
    'USDC' : r'../resources/data/raw/USDC_USDT_1m.csv',
    # ───── V ─────
    'VET'  : r'../resources/data/raw/VET_USDT_1m.csv',
    # ───── W ─────
    'WAN'  : r'../resources/data/raw/WAN_USDT_1m.csv',
    'WAVES': r'../resources/data/raw/WAVES_USDT_1m.csv',
    'WIN'  : r'../resources/data/raw/WIN_USDT_1m.csv',
    # ───── X ─────
    'XLM'  : r'../resources/data/raw/XLM_USDT_1m.csv',
    'XMR'  : r'../resources/data/raw/XMR_USDT_1m.csv',
    'XRP'  : r'../resources/data/raw/XRP_USDT_1m.csv',
    # ───── Z ─────
    'ZEC'  : r'../resources/data/raw/ZEC_USDT_1m.csv',
    'ZIL'  : r'../resources/data/raw/ZIL_USDT_1m.csv',
    'ZRX'  : r'../resources/data/raw/ZRX_USDT_1m.csv',
}

COL_PRICE    = 'close'      # ценовата колона в CSV-то
TIMEZONE     = 'UTC'        # всички в UTC
RISK_FREE    = 0.00         # годишен r_f; 0.0 ако не ти трябва
WEIGHT_CAP   = 0.50         # ≤ 50 % в един актив
N_PORTFOLIOS = 25_000       # Монте Карло проби
SEED         = 42
np.random.seed(SEED)

# --- 1-минутни барове → 525 600 на година ----------------------------
BAR_SEC           = 60
PERIODS_PER_YEAR  = int(365 * 86_400 / BAR_SEC) 
print(f'periods_per_year = {PERIODS_PER_YEAR:,}')

periods_per_year = 525,600


In [3]:
# ╔═════════════════════════════════════════════════════════════╗
# ║ 1) Зареждане на минутните цени                              ║
# ╚═════════════════════════════════════════════════════════════╝
def load_iso_csv(path: str | Path,
                 price_col: str = COL_PRICE,
                 tz: str = TIMEZONE) -> pd.Series:
    """
    Чете CSV с ISO `timestamp` и връща Series<price> (tz-aware).
    """
    df = pd.read_csv(path,
                     parse_dates=['timestamp'],
                     index_col='timestamp')
    df.index = df.index.tz_localize(tz)
    return df[price_col].astype(float)


# --- комбинираме всички активи ----------------------------------------
prices = pd.concat(
    {tkr: load_iso_csv(p) for tkr, p in FILEMAP.items()},
    axis=1,
    join='inner'                   # пазим минутите, налични за всички
).sort_index()

print(prices.head())

                              ADA    ALGO     ANKR   ATOM     BAT    BNB  \
timestamp                                                                  
2023-06-09 12:18:00+00:00  0.3184  0.1254  0.02374  9.368  0.1966  261.3   
2023-06-09 12:19:00+00:00  0.3186  0.1254  0.02376  9.377  0.1966  261.6   
2023-06-09 12:20:00+00:00  0.3185  0.1255  0.02379  9.377  0.1966  261.7   
2023-06-09 12:21:00+00:00  0.3196  0.1254  0.02375  9.373  0.1966  261.6   
2023-06-09 12:22:00+00:00  0.3204  0.1253  0.02373  9.371  0.1966  261.5   

                                BTC     CELR     CHZ      COS  ...      VET  \
timestamp                                                      ...            
2023-06-09 12:18:00+00:00  26640.01  0.01655  0.0880  0.00532  ...  0.01735   
2023-06-09 12:19:00+00:00  26654.99  0.01656  0.0880  0.00532  ...  0.01735   
2023-06-09 12:20:00+00:00  26661.16  0.01658  0.0881  0.00531  ...  0.01736   
2023-06-09 12:21:00+00:00  26643.89  0.01658  0.0881  0.00531  ...  0.01

In [4]:
# ╔═════════════════════════════════════════════════════════════╗
# ║ 2) Минутни доходности, μ̅, Σ                                ║
# ╚═════════════════════════════════════════════════════════════╝
rets = prices.pct_change().dropna()

# проверка, че са 1-минутни
freq = rets.index.to_series().diff().dt.total_seconds().median()
assert freq == BAR_SEC, f'Открих {freq=} сек. – не е 1-мин серия!'

means_bar = rets.mean().values                  # μ_i  (за 1 бар)
cov_bar   = rets.cov().values                  # Σ    (за 1 бар)
tickers   = list(FILEMAP.keys())

# кратка статистика
stats = pd.DataFrame({'μ_bar': means_bar,
                      'σ_bar': rets.std(ddof=0).values},
                     index=tickers)
stats['Sharpe_bar'] = stats['μ_bar'] / stats['σ_bar']
display(stats)


,μ_bar,σ_bar,Sharpe_bar
ADA,1.231035e-06,0.000947,0.001300
ALGO,1.101865e-06,0.001148,0.000960
ANKR,1.090526e-06,0.001130,0.000965
ATOM,4.359595e-07,0.000890,0.000490
BAT,1.040168e-06,0.000977,0.001065
BNB,-4.950530e-09,0.000669,-0.000007
BTC,1.568978e-06,0.000529,0.002966
CELR,3.242516e-07,0.001196,0.000271
CHZ,2.389043e-07,0.001237,0.000193
COS,1.251937e-06,0.001392,0.000900


In [5]:
# ╔═════════════════════════════════════════════════════════════╗
# ║ 3) Монте-Карло портфейли (с ограничение на теглото)         ║
# ╚═════════════════════════════════════════════════════════════╝
n_assets = len(tickers)
results  = np.zeros((3, N_PORTFOLIOS))          # σ, μ, Sharpe
weights  = np.zeros((N_PORTFOLIOS, n_assets))

i = 0
while i < N_PORTFOLIOS:
    w = np.random.random(n_assets)
    w /= w.sum()
    if (w > WEIGHT_CAP).any():          # концентрационен лимит
        continue

    mu_bar  = np.dot(w, means_bar)
    sig_bar = np.sqrt(w @ cov_bar @ w)

    mu_ann  = mu_bar  * PERIODS_PER_YEAR
    sig_ann = sig_bar * np.sqrt(PERIODS_PER_YEAR)

    weights[i]   = w
    results[0,i] = sig_ann
    results[1,i] = mu_ann
    results[2,i] = (mu_ann - RISK_FREE) / sig_ann

    i += 1

idx_max_sharpe = results[2].argmax()
idx_min_vol    = results[0].argmin()


In [6]:
# ╔═════════════════════════════════════════════════════════════╗
# ║ 4) Графика: Efficient Frontier                             ║
# ╚═════════════════════════════════════════════════════════════╝
fig, ax = plt.subplots(figsize=(9, 6))
sc = ax.scatter(results[0], results[1],
                c=results[2], cmap='YlGnBu', s=8, alpha=0.7)
fig.colorbar(sc, label='Sharpe ratio')

ax.scatter(*results[:2, idx_max_sharpe],
           marker='*', s=160, color='r', label='Max Sharpe')
ax.scatter(*results[:2, idx_min_vol],
           marker='X', s=160, color='g', label='Min volatility')

ax.set(title='Efficient Frontier – 1-min Crypto',
       xlabel='Annualised Volatility',
       ylabel='Annualised Return')
ax.legend(); plt.tight_layout()


In [7]:
# ╔═════════════════════════════════════════════════════════════╗
# ║ 5) Показване на разпределенията                             ║
# ╚═════════════════════════════════════════════════════════════╝
alloc_max = pd.Series(weights[idx_max_sharpe], index=tickers).round(2)
alloc_min = pd.Series(weights[idx_min_vol   ], index=tickers).round(2)

print('\n' + '─'*70)
print('ПОРТФЕЙЛ С МАКС. SHARPE')
print(f"  Год. доходност : {results[1, idx_max_sharpe]:.2%}")
print(f"  Год. волатилност : {results[0, idx_max_sharpe]:.2%}\n")
display(alloc_max.to_frame('Weight').T)

print('\n' + '─'*70)
print('ПОРТФЕЙЛ С МИН. ВОЛАТИЛНОСТ')
print(f"  Год. доходност : {results[1, idx_min_vol]:.2%}")
print(f"  Год. волатилност : {results[0, idx_min_vol]:.2%}\n")
display(alloc_min.to_frame('Weight').T)



──────────────────────────────────────────────────────────────────────
ПОРТФЕЙЛ С МАКС. SHARPE
  Год. доходност : 81.35%
  Год. волатилност : 39.91%



,ADA,ALGO,ANKR,ATOM,BAT,BNB,BTC,CELR,CHZ,COS,...,VET,WAN,WAVES,WIN,XLM,XMR,XRP,ZEC,ZIL,ZRX
Weight,0.01,0.0,0.0,0.02,0.03,0.02,0.04,0.02,0.0,0.02,...,0.01,0.03,0.03,0.03,0.03,0.01,0.01,0.01,0.01,0.03



──────────────────────────────────────────────────────────────────────
ПОРТФЕЙЛ С МИН. ВОЛАТИЛНОСТ
  Год. доходност : 69.26%
  Год. волатилност : 39.09%



,ADA,ALGO,ANKR,ATOM,BAT,BNB,BTC,CELR,CHZ,COS,...,VET,WAN,WAVES,WIN,XLM,XMR,XRP,ZEC,ZIL,ZRX
Weight,0.01,0.01,0.03,0.0,0.01,0.0,0.01,0.01,0.03,0.03,...,0.03,0.03,0.02,0.03,0.02,0.01,0.02,0.01,0.01,0.02


In [8]:
# ╔═════════════════════════════════════════════════════════════╗
# ║ 6) Генетичен алгоритъм – оптимизация на Sharpe              ║
# ╚═════════════════════════════════════════════════════════════╝
import random

# --------------------------------------------------------------
# Хиперпараметри на GA
# --------------------------------------------------------------
POP_SIZE       = 1000          # колко индивида във всяко поколение
NB_GENERATIONS = 50             # колко поколения
ELITE_FRAC     = 0.10           # най-добрият % индивиди, които копираме директно
MUTATE_FRAC    = 0.15           # шанс дадено дете да бъде мутант
MUTATE_SCALE   = 0.10           # колко силно „рита“ мутантът теглото
PLOT_X = 5
PLOT_Y = 5


np.random.seed(123)

# --------------------------------------------------------------
# Фитнес функция – годишен Sharpe
# --------------------------------------------------------------
def fitness(w: np.ndarray) -> float:
    """
    w трябва вече да е нормализиран: Σ w_i = 1
    """
    mu_bar  = w @ means_bar
    sig_bar = np.sqrt(w @ cov_bar @ w)
    
    mu_ann  = mu_bar  * PERIODS_PER_YEAR
    sig_ann = sig_bar * np.sqrt(PERIODS_PER_YEAR)
    return (mu_ann - RISK_FREE) / sig_ann


# --------------------------------------------------------------
# GA инструменти: кросоувър, мутация, създаване на поколение
# --------------------------------------------------------------
def crossover(p1: np.ndarray, p2: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Симпъл α-crossover (α ~ U[0,1])
    """
    α = np.random.rand()
    c1 = α * p1 + (1 - α) * p2
    c2 = (1 - α) * p1 + α * p2
    return c1, c2

def mutate(child: np.ndarray) -> np.ndarray:
    """
    Избутва случайно тегло с N(0, MUTATE_SCALE).
    После нормализира и реже до WEIGHT_CAP.
    """
    idx = np.random.randint(len(child))
    child[idx] += np.random.normal(0, MUTATE_SCALE)
    child = np.clip(child, 0, WEIGHT_CAP)      # защитно изрязване
    child /= child.sum()
    return child

def next_generation(pop: list[np.ndarray]) -> list[np.ndarray]:
    """
    Създава следващото поколение:
      1. сортира по fitness (низходящ)
      2. взема ELITE_FRAC като “елит”
      3. прави деца чрез кросоувър
      4. мутира част от децата
      5. нормализира теглата
    """
    pop = sorted(pop, key=fitness, reverse=True)
    elite_n = int(len(pop) * ELITE_FRAC)
    new_pop = pop[:elite_n]                       # запазваме елита
    
    # ––– рандомизираме елита, за да правим случайни двойки
    random.shuffle(new_pop)
    parents = new_pop.copy()
    
    # ––– кросоувър
    children = []
    for p1, p2 in zip(parents[::2], parents[1::2]):
        c1, c2 = crossover(p1, p2)
        children.extend([c1, c2])
    
    # ––– мутации
    for k in range(len(children)):
        if np.random.rand() < MUTATE_FRAC:
            children[k] = mutate(children[k])
        # задължителна нормализация + cap
        children[k] = np.clip(children[k], 0, WEIGHT_CAP)
        children[k] /= children[k].sum()
    
    new_pop.extend(children)
    
    # ако популацията е недостатъчна (нечетен брой родители)
    while len(new_pop) < POP_SIZE:
        w = np.random.random(len(tickers))
        w /= w.sum()
        if (w > WEIGHT_CAP).any():
            continue
        new_pop.append(w)
    return new_pop[:POP_SIZE]


# hyperparameters optimizer
import numpy as np, random, math, optuna, time
from copy import deepcopy
def run_ga_once(pop_size, nb_generations, elite_frac,
                mutate_frac, mutate_scale,
                weight_cap, seed=None):
    """
    Стартира ЕДНО изпълнение на GA с подадените хиперпараметри.
    Връща Sharpe на най-добрия индивид.
    """

    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # -------------- локални override-и на глобалните константи --------------
    global POP_SIZE, NB_GENERATIONS, ELITE_FRAC
    global MUTATE_FRAC, MUTATE_SCALE, WEIGHT_CAP

    POP_SIZE       = pop_size
    NB_GENERATIONS = nb_generations
    ELITE_FRAC     = elite_frac
    MUTATE_FRAC    = mutate_frac
    MUTATE_SCALE   = mutate_scale
    WEIGHT_CAP     = weight_cap          # нов параметър!

    # -------------- инициализираме първото поколение ------------------------
    population = []
    while len(population) < POP_SIZE:
        w = np.random.random(len(tickers))
        w /= w.sum()
        if (w > WEIGHT_CAP).any():
            continue
        population.append(w)

    # -------------- еволюираме ----------------------------------------------
    best_fit = -math.inf
    for g in range(NB_GENERATIONS):
        population = next_generation(population)
        # оценяваме елита
        top = max(population, key=fitness)
        best_fit = max(best_fit, fitness(top))

    return best_fit   # ↑ Sharpe на най-силния индивид

def objective(trial: optuna.trial.Trial) -> float:
    """
    Optuna ще пробва различни GA-хиперпараметри и ще
    връща >>средния<< Sharpe от K рестарта на GA.
    """
    # 1) дефинираме търсено пространство
    pop_size       = trial.suggest_int   ('pop_size',        200, 1500, step=100)
    nb_generations = trial.suggest_int   ('nb_generations',   20, 200,  step=10)
    elite_frac     = trial.suggest_float ('elite_frac',      0.05, 0.35)
    mutate_frac    = trial.suggest_float ('mutate_frac',     0.05, 0.40)
    mutate_scale   = trial.suggest_float ('mutate_scale',    0.01, 0.50, log=True)
    weight_cap     = trial.suggest_float ('weight_cap',      0.30, 0.70)

    # 2) за стабилност ‒ стартираме GA K пъти с разл. seed
    K, scores = 3, []
    for k in range(K):
        seed = 2024 + k
        score = run_ga_once(pop_size, nb_generations,
                            elite_frac, mutate_frac, mutate_scale,
                            weight_cap, seed=seed)
        scores.append(score)

    # 3) Optuna максимизира -> връщаме средната стойност
    return float(np.mean(scores))

study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(),
                            pruner =optuna.pruners.MedianPruner(
                                        n_startup_trials=10, n_warmup_steps=5))
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
study.optimize(objective,
               n_trials = 40,          # или по време: timeout=3600
               show_progress_bar=True)

print("🏆 Най-добрите GA-хиперпараметри:")
for k, v in study.best_trial.params.items():
    print(f"  {k:15s} : {v}")
print("Среден Sharpe  :", study.best_value)




/home/npenchev/fun/summer-school-2025-crypto-forecasting/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-27 21:15:46,180] A new study created in memory with name: no-name-4626f565-70c1-4bd1-a979-17bfa9af4a6f
Best trial: 0. Best value: 2.92911:   2%|██▉                                                                                                                 | 1/40 [00:02<01:20,  2.06s/it]

[I 2025-06-27 21:15:48,238] Trial 0 finished with value: 2.9291122642338845 and parameters: {'pop_size': 200, 'nb_generations': 110, 'elite_frac': 0.2810582153946935, 'mutate_frac': 0.3154039449640288, 'mutate_scale': 0.025685240011970218, 'weight_cap': 0.6569451311484661}. Best is trial 0 with value: 2.9291122642338845.


Best trial: 1. Best value: 2.97768:   5%|█████▊                                                                                                              | 2/40 [00:04<01:33,  2.46s/it]

[I 2025-06-27 21:15:50,976] Trial 1 finished with value: 2.977679766358303 and parameters: {'pop_size': 400, 'nb_generations': 80, 'elite_frac': 0.3074368749751508, 'mutate_frac': 0.09155583683938934, 'mutate_scale': 0.08027123871366869, 'weight_cap': 0.602287445155194}. Best is trial 1 with value: 2.977679766358303.


Best trial: 1. Best value: 2.97768:   8%|████████▋                                                                                                           | 3/40 [00:06<01:16,  2.06s/it]

...

[I 2025-06-27 21:26:38,941] Trial 37 finished with value: 3.1460875682581917 and parameters: {'pop_size': 1400, 'nb_generations': 60, 'elite_frac': 0.11733419002991224, 'mutate_frac': 0.3735050925918108, 'mutate_scale': 0.09165512152387843, 'weight_cap': 0.5904593414079071}. Best is trial 26 with value: 3.645995828335336.


Best trial: 26. Best value: 3.646:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 39/40 [11:20<00:22, 22.71s/it]

[I 2025-06-27 21:27:06,279] Trial 38 finished with value: 2.9832442078995385 and parameters: {'pop_size': 1200, 'nb_generations': 190, 'elite_frac': 0.1366042628500793, 'mutate_frac': 0.27251898708511957, 'mutate_scale': 0.010705169225202557, 'weight_cap': 0.646605689579}. Best is trial 26 with value: 3.645995828335336.


Best trial: 26. Best value: 3.646: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [11:39<00:00, 17.48s/it]

[I 2025-06-27 21:27:25,569] Trial 39 finished with value: 3.538022857365838 and parameters: {'pop_size': 1000, 'nb_generations': 170, 'elite_frac': 0.11267965292644117, 'mutate_frac': 0.31872861026232696, 'mutate_scale': 0.06873423978048006, 'weight_cap': 0.3945011607171553}. Best is trial 26 with value: 3.645995828335336.
🏆 Най-добрите GA-хиперпараметри:
  pop_size        : 1300
  nb_generations  : 200
  elite_frac      : 0.21950893438521896
  mutate_frac     : 0.3824611821082534
  mutate_scale    : 0.1669082947874534
  weight_cap      : 0.37600756995554485
Среден Sharpe  : 3.645995828335336


In [9]:
    # POP_SIZE       = pop_size
    # NB_GENERATIONS = nb_generations
    # ELITE_FRAC     = elite_frac
    # MUTATE_FRAC    = mutate_frac
    # MUTATE_SCALE   = mutate_scale
    # WEIGHT_CAP     = weight_cap 
print("OPTIMIZED HYPERPARAMETERS")
print(f"POP_SIZE: {POP_SIZE}")
print(f"NB_GENERATIONS: {NB_GENERATIONS}")
print(f"ELITE_FRAC: {ELITE_FRAC}")
print(f"MUTATE_FRAC: {MUTATE_FRAC}")
print(f"MUTATE_SCALE: {MUTATE_SCALE}")
print(f"WEIGHT_CAP: {WEIGHT_CAP}")

OPTIMIZED HYPERPARAMETERS
POP_SIZE: 1000
NB_GENERATIONS: 170
ELITE_FRAC: 0.11267965292644117
MUTATE_FRAC: 0.31872861026232696
MUTATE_SCALE: 0.06873423978048006
WEIGHT_CAP: 0.3945011607171553


In [10]:
# --------------------------------------------------------------
# 6.1) Инициализация на първото поколение
# --------------------------------------------------------------
population = []
while len(population) < POP_SIZE:
    w = np.random.random(len(tickers))
    w /= w.sum()
    if (w > WEIGHT_CAP).any():
        continue
    population.append(w)

# --------------------------------------------------------------
# 6.2) Еволюция и визуализация
# --------------------------------------------------------------
# fig, axs = plt.subplots(PLOT_X, PLOT_Y, figsize=(15, 13), sharex=True, sharey=True)
# axs = axs.flatten()

fig, axs = plt.subplots(PLOT_Y, PLOT_X, figsize=(15, 13),
                        sharex=True, sharey=True)
axs = axs.flatten()
MAX_PLOTS = PLOT_X * PLOT_Y      # 25

# кои поколения ще рисуваме
if NB_GENERATIONS <= MAX_PLOTS:
    gens_to_plot = list(range(NB_GENERATIONS))
else:
    # равномерни индекси, винаги включва 0
    gens_to_plot = (np.linspace(0, NB_GENERATIONS - 1,
                                num=MAX_PLOTS, dtype=int)
                      .tolist())

print("Ще се визуализират поколения:", gens_to_plot)

for g in range(NB_GENERATIONS):
    # ── статистика за текущото поколение ───────────────────────
    res = np.zeros((2, POP_SIZE))            # σ, μ  (годишни)
    for i, w in enumerate(population):
        mu_bar  = w @ means_bar
        sig_bar = np.sqrt(w @ cov_bar @ w)
        res[1, i] = mu_bar  * PERIODS_PER_YEAR
        res[0, i] = sig_bar * np.sqrt(PERIODS_PER_YEAR)

    # ── чертаем само ако g e в списъка ──────────────────────────
    if g in gens_to_plot:
        plot_idx = gens_to_plot.index(g)     # 0 … 24
        ax = axs[plot_idx]
        ax.scatter(res[0], res[1], s=6, alpha=0.5, c='steelblue')
        ax.set_title(f'Gen {g+1}')
        ax.set_xlabel('σ (ann)')
        ax.set_ylabel('μ (ann)')

    # ── еволюираме към следващо поколение ───────────────────────
    population = next_generation(population)

plt.suptitle('GA evolution toward Efficient Frontier', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

Ще се визуализират поколения: [0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 169]


/tmp/ipykernel_218925/3415149344.py:57: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
TOP_N = 20        # how many best portfolios you want to list

def ann_stats(w: np.ndarray) -> tuple[float, float, float]:
    mu_bar  = w @ means_bar
    sig_bar = np.sqrt(w @ cov_bar @ w)
    mu_ann  = mu_bar  * PERIODS_PER_YEAR
    sig_ann = sig_bar * np.sqrt(PERIODS_PER_YEAR)
    sharpe  = (mu_ann - RISK_FREE) / sig_ann
    return mu_ann, sig_ann, sharpe

# 1) take the best TOP_N by Sharpe
best_pop = sorted(population, key=fitness, reverse=True)[:TOP_N]

# 2) build one combined table
records = []
for rank, w in enumerate(best_pop, 1):
    mu, sig, shp = ann_stats(w)
    rec = {
        'Rank'      : rank,
        'Annual μ'  : round(mu, 4),
        'Annual σ'  : round(sig, 4),
        'Sharpe'    : round(shp, 3)
    }
    # add the weights, rounded to 3 dp
    rec.update({tkr: round(w[i], 3) for i, tkr in enumerate(tickers)})
    records.append(rec)

top_df = (pd.DataFrame(records)
            .set_index('Rank')
            .sort_index())

display(top_df)


,Annual μ,Annual σ,Sharpe,ADA,ALGO,ANKR,ATOM,BAT,BNB,BTC,...,VET,WAN,WAVES,WIN,XLM,XMR,XRP,ZEC,ZIL,ZRX
Rank,,,,,,,,,,,,,,,,,,,,,
1,1.4989,0.4251,3.526,0.004,0.002,0.002,0.001,0.002,0.002,0.013,...,0.004,0.018,0.006,0.006,0.005,0.005,0.003,0.002,0.003,0.064
2,1.5038,0.4265,3.526,0.003,0.002,0.002,0.002,0.002,0.002,0.014,...,0.004,0.019,0.006,0.006,0.005,0.005,0.003,0.002,0.003,0.065
3,1.4996,0.4255,3.524,0.003,0.002,0.002,0.001,0.001,0.002,0.014,...,0.004,0.017,0.006,0.006,0.005,0.005,0.003,0.002,0.003,0.066
4,1.5010,0.4260,3.524,0.004,0.002,0.002,0.002,0.002,0.002,0.014,...,0.004,0.018,0.006,0.006,0.005,0.005,0.002,0.002,0.003,0.065
5,1.4977,0.4250,3.524,0.001,0.002,0.002,0.002,0.002,0.002,0.013,...,0.003,0.023,0.006,0.006,0.005,0.005,0.002,0.002,0.003,0.070
6,1.4992,0.4255,3.523,0.003,0.002,0.002,0.001,0.001,0.002,0.014,...,0.004,0.018,0.006,0.006,0.005,0.005,0.003,0.002,0.003,0.066
7,1.5019,0.4263,3.523,0.004,0.002,0.002,0.002,0.002,0.000,0.014,...,0.004,0.018,0.006,0.006,0.005,0.005,0.002,0.002,0.003,0.066
8,1.4984,0.4253,3.523,0.004,0.002,0.002,0.002,0.002,0.002,0.013,...,0.003,0.020,0.006,0.006,0.005,0.005,0.003,0.002,0.003,0.064
9,1.5048,0.4272,3.523,0.004,0.002,0.002,0.002,0.002,0.002,0.014,...,0.004,0.017,0.006,0.006,0.005,0.005,0.002,0.002,0.003,0.067


In [12]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.scatter(top_df['Annual σ'], top_df['Annual μ'],
            s=90, color='steelblue', edgecolors='k', alpha=0.8)

# (по желание) етикет към всяка точка – номерът в класацията
for rank, row in top_df.iterrows():
    plt.annotate(str(row['Sharpe']),
                 (row['Annual σ'], row['Annual μ']),
                 textcoords="offset points",
                 xytext=(4, 4), ha='left', fontsize=9)

plt.xlabel('Annualised σ (volatility)')
plt.ylabel('Annualised μ (return)')
plt.title(f'TOP {len(top_df)} portfolios – risk / return space')
plt.grid(True, ls='--', alpha=0.4)
plt.tight_layout()
plt.show()

/tmp/ipykernel_218925/3338961641.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
best_w = top_df.iloc[0][tickers].to_dict()
best_w

# ПОРТФЕЙЛ С МАКС. SHARPE
#   Год. доходност : 98.79%
#   Год. волатилност : 60.37%

# BTC	ADA	ALGO	ANKR
# Weight	0.5	0.11	0.09	0.3

# ──────────────────────────────────────────────────────────────────────
# ПОРТФЕЙЛ С МИН. ВОЛАТИЛНОСТ
#   Год. доходност : 90.08%
#   Год. волатилност : 57.66%

# BTC	ADA	ALGO	ANKR
# Weight	0.5	0.31	0.12	0.07

{'ADA': 0.004,
 'ALGO': 0.002,
 'ANKR': 0.002,
 'ATOM': 0.001,
 'BAT': 0.002,
 'BNB': 0.002,
 'BTC': 0.013,
 'CELR': 0.0,
 'CHZ': 0.002,
 'COS': 0.026,
 'CVC': 0.06,
 'DASH': 0.001,
 'DENT': 0.002,
 'DOCK': 0.046,
 'DOGE': 0.003,
 'DUSK': 0.001,
 'ENJ': 0.002,
 'EOS': 0.001,
 'ETC': 0.002,
 'ETH': 0.006,
 'FET': 0.069,
 'FTM': 0.002,
 'FUN': 0.035,
 'HOT': 0.002,
 'ICX': 0.003,
 'IOST': 0.003,
 'IOTA': 0.003,
 'KEY': 0.002,
 'LINK': 0.238,
 'LTC': 0.001,
 'MATIC': 0.002,
 'MTL': 0.006,
 'NEO': 0.003,
 'NULS': 0.005,
 'OMG': 0.002,
 'ONE': 0.002,
 'ONG': 0.083,
 'ONT': 0.003,
 'PERL': 0.036,
 'QTUM': 0.005,
 'TFUEL': 0.019,
 'THETA': 0.005,
 'TOMO': 0.003,
 'TRX': 0.156,
 'TUSD': 0.009,
 'USDC': 0.009,
 'VET': 0.004,
 'WAN': 0.018,
 'WAVES': 0.006,
 'WIN': 0.006,
 'XLM': 0.005,
 'XMR': 0.005,
 'XRP': 0.003,
 'ZEC': 0.002,
 'ZIL': 0.003,
 'ZRX': 0.064}

In [14]:
# -*- coding: utf-8 -*-
"""
Backtrader static-weight portfolio back-test
"""
import matplotlib
matplotlib.use('Agg')  
import backtrader as bt
import pandas as pd
from pathlib import Path


# ---------- Параметри ------------------------------------------------------

WEIGHTS = best_w

DATA_DIR       = Path('../resources/data/raw')   # коригирай пътя, ако е нужно
STARTING_CASH  = 100_000
COMMISSION     = 0.001        # 0.1 %
REBALANCE_DAYS = 30         # напр. 30 за месечен ребаланс


# ---------- Стратегия ------------------------------------------------------

class StaticWeightStrategy(bt.Strategy):
    """
    Поддържа фиксирани тегла. Ребалансира:
      • веднъж на първата свещ;
      • през `rebalance_days`, ако е зададено.
    """
    params = (
        ('weights',         None),
        ('rebalance_days',  None),
    )

    def __init__(self):
        self.rebalanced_init = False     # ще ребалансираме при първата свещ
        self.last_rebalance  = None      # календарно следене (ако е нужно)

    # --------------------------------------------------
    def next(self):
        dt = self.datas[0].datetime.date(0)   # текуща дата

        # --- първоначален ребаланс ---------------------------------
        if not self.rebalanced_init:
            self.rebalance()
            self.rebalanced_init = True
            self.last_rebalance  = dt
            return                          # излизаме, за да не удвоим ребаланса

        # --- периодичен ребаланс -----------------------------------
        if self.p.rebalance_days:
            if (dt - self.last_rebalance).days >= self.p.rebalance_days:
                self.rebalance()
                self.last_rebalance = dt

    # --------------------------------------------------
    def rebalance(self):
        """
        Изравняване на позициите към зададените тегла.
        """
        port_value = self.broker.getvalue()
        for data in self.datas:
            w = self.p.weights.get(data._name, 0.0)
            self.order_target_percent(data=data, target=w)

    # --------------------------------------------------
    def log(self, txt, dt=None):
        pass  # махни 'pass' и сложи print, ако искаш лог


# ---------- Четене на CSV → DataFeed --------------------------------------

def load_feed(path: str | Path, name: str) -> bt.feeds.PandasData:
    """
    Чете 1-minute Binance CSV → ресемплира към дневни свещи.
    """
    df = (
        pd.read_csv(path, parse_dates=['timestamp'], index_col='timestamp')
          .sort_index()
          .resample('1D')
          .agg({'open':'first', 'high':'max', 'low':'min',
                'close':'last', 'volume':'sum'})
          .dropna()
    )

    print(f'{name}: {len(df):>4} дни | {df.index.min().date()} → {df.index.max().date()}')

    return bt.feeds.PandasData(
        dataname=df,
        name=name,
        timeframe=bt.TimeFrame.Days,
        compression=1
    )


# ---------- Cerebro pipeline ----------------------------------------------

cerebro = bt.Cerebro()

# 1) стратегия
cerebro.addstrategy(
    StaticWeightStrategy,
    weights=WEIGHTS,
    rebalance_days=REBALANCE_DAYS
)

# 2) брокер
cerebro.broker.setcash(STARTING_CASH)
cerebro.broker.setcommission(commission=COMMISSION)
#  ➜  BUY/SELL стрелки
cerebro.addobserver(bt.observers.BuySell)     # 📈 показва мястото на всеки ордер
cerebro.addobserver(bt.observers.Trades)      # 💬 балони с резултата от сделката

# 3) данни
cerebro.adddata(load_feed(DATA_DIR / 'BTC_USDT_1m.csv',  'BTC'))
cerebro.adddata(load_feed(DATA_DIR / 'ADA_USDT_1m.csv',   'ADA'))
cerebro.adddata(load_feed(DATA_DIR / 'ALGO_USDT_1m.csv',  'ALGO'))
cerebro.adddata(load_feed(DATA_DIR / 'ANKR_USDT_1m.csv',  'ANKR'))
cerebro.adddata(load_feed(DATA_DIR / 'ATOM_USDT_1m.csv',  'ATOM'))
cerebro.adddata(load_feed(DATA_DIR / 'BAT_USDT_1m.csv',   'BAT'))
cerebro.adddata(load_feed(DATA_DIR / 'BNB_USDT_1m.csv',   'BNB'))
cerebro.adddata(load_feed(DATA_DIR / 'CELR_USDT_1m.csv',  'CELR'))
cerebro.adddata(load_feed(DATA_DIR / 'CHZ_USDT_1m.csv',   'CHZ'))
cerebro.adddata(load_feed(DATA_DIR / 'COS_USDT_1m.csv',   'COS'))
cerebro.adddata(load_feed(DATA_DIR / 'CVC_USDT_1m.csv',   'CVC'))
cerebro.adddata(load_feed(DATA_DIR / 'DASH_USDT_1m.csv',  'DASH'))
cerebro.adddata(load_feed(DATA_DIR / 'DENT_USDT_1m.csv',  'DENT'))
cerebro.adddata(load_feed(DATA_DIR / 'DOCK_USDT_1m.csv',  'DOCK'))
cerebro.adddata(load_feed(DATA_DIR / 'DOGE_USDT_1m.csv',  'DOGE'))
cerebro.adddata(load_feed(DATA_DIR / 'DUSK_USDT_1m.csv',  'DUSK'))
cerebro.adddata(load_feed(DATA_DIR / 'ENJ_USDT_1m.csv',   'ENJ'))
cerebro.adddata(load_feed(DATA_DIR / 'EOS_USDT_1m.csv',   'EOS'))
cerebro.adddata(load_feed(DATA_DIR / 'ETC_USDT_1m.csv',   'ETC'))
cerebro.adddata(load_feed(DATA_DIR / 'ETH_USDT_1m.csv',   'ETH'))
cerebro.adddata(load_feed(DATA_DIR / 'FET_USDT_1m.csv',   'FET'))
cerebro.adddata(load_feed(DATA_DIR / 'FTM_USDT_1m.csv',   'FTM'))
cerebro.adddata(load_feed(DATA_DIR / 'FUN_USDT_1m.csv',   'FUN'))
cerebro.adddata(load_feed(DATA_DIR / 'HOT_USDT_1m.csv',   'HOT'))
cerebro.adddata(load_feed(DATA_DIR / 'ICX_USDT_1m.csv',   'ICX'))
cerebro.adddata(load_feed(DATA_DIR / 'IOST_USDT_1m.csv',  'IOST'))
cerebro.adddata(load_feed(DATA_DIR / 'IOTA_USDT_1m.csv',  'IOTA'))
cerebro.adddata(load_feed(DATA_DIR / 'KEY_USDT_1m.csv',   'KEY'))
cerebro.adddata(load_feed(DATA_DIR / 'LINK_USDT_1m.csv',  'LINK'))
cerebro.adddata(load_feed(DATA_DIR / 'LTC_USDT_1m.csv',   'LTC'))
cerebro.adddata(load_feed(DATA_DIR / 'MATIC_USDT_1m.csv', 'MATIC'))
cerebro.adddata(load_feed(DATA_DIR / 'MTL_USDT_1m.csv',   'MTL'))
cerebro.adddata(load_feed(DATA_DIR / 'NEO_USDT_1m.csv',   'NEO'))
cerebro.adddata(load_feed(DATA_DIR / 'NULS_USDT_1m.csv',  'NULS'))
cerebro.adddata(load_feed(DATA_DIR / 'OMG_USDT_1m.csv',   'OMG'))
cerebro.adddata(load_feed(DATA_DIR / 'ONE_USDT_1m.csv',   'ONE'))
cerebro.adddata(load_feed(DATA_DIR / 'ONG_USDT_1m.csv',   'ONG'))
cerebro.adddata(load_feed(DATA_DIR / 'ONT_USDT_1m.csv',   'ONT'))
cerebro.adddata(load_feed(DATA_DIR / 'PERL_USDT_1m.csv',  'PERL'))
cerebro.adddata(load_feed(DATA_DIR / 'QTUM_USDT_1m.csv',  'QTUM'))
cerebro.adddata(load_feed(DATA_DIR / 'TFUEL_USDT_1m.csv', 'TFUEL'))
cerebro.adddata(load_feed(DATA_DIR / 'THETA_USDT_1m.csv', 'THETA'))
cerebro.adddata(load_feed(DATA_DIR / 'TOMO_USDT_1m.csv',  'TOMO'))
cerebro.adddata(load_feed(DATA_DIR / 'TRX_USDT_1m.csv',   'TRX'))
cerebro.adddata(load_feed(DATA_DIR / 'TUSD_USDT_1m.csv',  'TUSD'))
cerebro.adddata(load_feed(DATA_DIR / 'USDC_USDT_1m.csv',  'USDC'))
cerebro.adddata(load_feed(DATA_DIR / 'VET_USDT_1m.csv',   'VET'))
cerebro.adddata(load_feed(DATA_DIR / 'WAN_USDT_1m.csv',   'WAN'))
cerebro.adddata(load_feed(DATA_DIR / 'WAVES_USDT_1m.csv', 'WAVES'))
cerebro.adddata(load_feed(DATA_DIR / 'WIN_USDT_1m.csv',   'WIN'))
cerebro.adddata(load_feed(DATA_DIR / 'XLM_USDT_1m.csv',   'XLM'))
cerebro.adddata(load_feed(DATA_DIR / 'XMR_USDT_1m.csv',   'XMR'))
cerebro.adddata(load_feed(DATA_DIR / 'XRP_USDT_1m.csv',   'XRP'))
cerebro.adddata(load_feed(DATA_DIR / 'ZEC_USDT_1m.csv',   'ZEC'))
cerebro.adddata(load_feed(DATA_DIR / 'ZIL_USDT_1m.csv',   'ZIL'))
cerebro.adddata(load_feed(DATA_DIR / 'ZRX_USDT_1m.csv',   'ZRX'))

# ---------- Бектест --------------------------------------------------------

print(f'\n🟡  Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}')
cerebro.run()
print(f'🟢  Final Portfolio Value:    {cerebro.broker.getvalue():,.2f}')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 20)
figs = cerebro.plot(iplot=True,                 # True = изкарва директно в cell-а
             style='candlestick',
             volume=False,
             barup='lime', bardown='red')
fig = figs[0][0]                              # първият (и единствен) прозорец
fig.savefig('backtest_chart.png', dpi=300)    # => ./backtest_chart.png
print('Графиката е записана в backtest_chart.png')
# cerebro.plot(style='candlestick')   # махни коментара за графика


BTC:  366 дни | 2023-06-09 → 2024-06-08
ADA:  366 дни | 2023-06-09 → 2024-06-08
ALGO:  366 дни | 2023-06-09 → 2024-06-08
ANKR:  366 дни | 2023-06-09 → 2024-06-08
ATOM:  366 дни | 2023-06-09 → 2024-06-08
BAT:  366 дни | 2023-06-09 → 2024-06-08
BNB:  366 дни | 2023-06-09 → 2024-06-08
CELR:  366 дни | 2023-06-09 → 2024-06-08
CHZ:  367 дни | 2023-06-09 → 2024-06-09
COS:  366 дни | 2023-06-09 → 2024-06-08
CVC:  367 дни | 2023-06-09 → 2024-06-09
DASH:  366 дни | 2023-06-09 → 2024-06-08
DENT:  366 дни | 2023-06-09 → 2024-06-08
DOCK:  367 дни | 2023-06-09 → 2024-06-09
DOGE:  366 дни | 2023-06-09 → 2024-06-08
DUSK:  366 дни | 2023-06-09 → 2024-06-08
ENJ:  366 дни | 2023-06-09 → 2024-06-08
EOS:  366 дни | 2023-06-09 → 2024-06-08
ETC:  366 дни | 2023-06-09 → 2024-06-08
ETH:  366 дни | 2023-06-09 → 2024-06-08
FET:  366 дни | 2023-06-09 → 2024-06-08
FTM:  366 дни | 2023-06-09 → 2024-06-08
FUN:  367 дни | 2023-06-09 → 2024-06-09
HOT:  366 дни | 2023-06-09 → 2024-06-08
ICX:  366 дни | 2023-06-09 → 20

/home/npenchev/fun/summer-school-2025-crypto-forecasting/.venv/lib64/python3.12/site-packages/backtrader/plot/plot.py:821: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  self.mpyplot.show()


<IPython.core.display.Javascript object>

Графиката е записана в backtest_chart.png
